In [1]:
import pandas as pd
import os
import glob
import duckdb
from fitparse import FitFile


In [ ]:

# --- CONFIGURATION ---

DB_FILE = 'project_database.duckdb'
ACTIVITIES_FILE = '../strava_data_dumps/STRAVA+export_8029714/activities.csv'
FIT_FILES_FOLDER = '../strava_data_dumps/STRAVA+export_8029714/activities/'

IMPORTANT_FIELDS = ['heart_rate', 'cadence', 'speed']


In [3]:

# Connect to DuckDB
con = duckdb.connect(DB_FILE)


In [4]:

# --- HELPER FUNCTIONS ---

def create_tables():
    """Create activities and fit_logs tables (DuckDB compatible)."""
    con.execute("DROP TABLE IF EXISTS activities")
    con.execute("DROP TABLE IF EXISTS fit_logs")

    con.execute("""
        CREATE TABLE activities (
            activity_id BIGINT,
            name TEXT,
            date TIMESTAMP,
            distance DOUBLE,
            duration DOUBLE,
            filename TEXT
        )
    """)

    con.execute("""
        CREATE TABLE fit_logs (
            log_id BIGINT,
            activity_id BIGINT,
            timestamp TIMESTAMP,
            field_name TEXT,
            field_value DOUBLE
        )
    """)
    print("✅ Tables created fresh.")

def get_next_id(table_name, id_column):
    """Find the next available ID for a table."""
    result = con.execute(f"SELECT MAX({id_column}) FROM {table_name}").fetchone()[0]
    return 1 if result is None else result + 1

def bulk_insert_activities(csv_path):
    """Load activities from CSV, assign activity_ids, and bulk insert."""
    df = pd.read_csv(csv_path)

    if 'Filename' not in df.columns:
        raise ValueError("activities.csv must have a 'filename' column.")

    next_id = get_next_id('activities', 'activity_id')
    df.insert(0, 'activity_id', range(next_id, next_id + len(df)))

    con.register('temp_activities', df)
    con.execute("INSERT INTO activities SELECT * FROM temp_activities")

    print(f"✅ Inserted {len(df)} activities starting at ID {next_id}")

def parse_fit_file(filepath):
    """Parse a .fit file into a list of records."""
    try:
        fitfile = FitFile(filepath)
    except Exception as e:
        print(f"⚠️ Failed to parse {filepath}: {e}")
        return []

    entries = []
    for record in fitfile.get_messages('record'):
        timestamp = None
        metrics = []

        for field in record:
            if field.name == 'timestamp':
                timestamp = field.value
            elif field.name in IMPORTANT_FIELDS and field.value is not None:
                metrics.append((field.name, field.value))

        if timestamp:
            for field_name, field_value in metrics:
                entries.append({
                    'timestamp': timestamp,
                    'field_name': field_name,
                    'field_value': field_value
                })

    return entries

def build_all_fit_logs():
    """Parse all .fit files into one big DataFrame of logs."""
    activities_df = pd.read_sql("SELECT activity_id, filename FROM activities", con)

    all_logs = []

    fit_files = glob.glob(os.path.join(FIT_FILES_FOLDER, '*.fit'))

    for filepath in fit_files:
        filename = os.path.basename(filepath)
        match = activities_df[activities_df['Filename'] == filename]

        if match.empty:
            print(f"⚠️ No matching activity for {filename}")
            continue

        activity_id = match.iloc[0]['activity_id']
        logs = parse_fit_file(filepath)

        if not logs:
            continue

        for log in logs:
            log['activity_id'] = activity_id

        all_logs.extend(logs)

    print(f"✅ Parsed {len(all_logs)} total log entries from FIT files.")
    return pd.DataFrame(all_logs)

def bulk_insert_fit_logs(df_logs):
    """Bulk insert fit logs with auto-incremented log_ids."""
    if df_logs.empty:
        print("⚠️ No logs to insert.")
        return

    next_id = get_next_id('fit_logs', 'log_id')
    df_logs.insert(0, 'log_id', range(next_id, next_id + len(df_logs)))

    con.register('temp_fit_logs', df_logs)
    con.execute("INSERT INTO fit_logs SELECT * FROM temp_fit_logs")

    print(f"✅ Inserted {len(df_logs)} logs starting at ID {next_id}")

def test_query():
    """Query activities joined with fit_logs."""
    df = con.execute("""
        SELECT a.name, f.timestamp, f.field_name, f.field_value
        FROM activities a
        JOIN fit_logs f ON a.activity_id = f.activity_id
        ORDER BY a.activity_id, f.timestamp
        LIMIT 10
    """).fetchdf()
    print("\n🎯 Sample joined data:")
    print(df)


In [5]:

# --- MAIN FLOW ---

if __name__ == "__main__":
    create_tables()
    bulk_insert_activities(ACTIVITIES_FILE)
    logs_df = build_all_fit_logs()
    bulk_insert_fit_logs(logs_df)
    test_query()
    print("\n🏁 All done!")


✅ Tables created fresh.


FileNotFoundError: [Errno 2] No such file or directory: 'strava_data_dumps/STRAVA+export_8029714/activities.csv'

# Basic DuckDB example -- Works

In [ ]:
import duckdb

# Connect to a DuckDB database file (it will create it if it doesn't exist)
con = duckdb.connect('test_project.duckdb')

# Drop tables if they exist (to start fresh)
con.execute("DROP TABLE IF EXISTS activities")
con.execute("DROP TABLE IF EXISTS fit_logs")

# Create activities table (MINIMAL, NO constraints)
con.execute("""
    CREATE TABLE activities (
        activity_id BIGINT,
        name TEXT,
        date TIMESTAMP,
        distance DOUBLE,
        duration DOUBLE,
        fit_filename TEXT
    )
""")

# Create fit_logs table (also MINIMAL)
con.execute("""
    CREATE TABLE fit_logs (
        log_id BIGINT,
        activity_id BIGINT,
        timestamp TIMESTAMP,
        field_name TEXT,
        field_value DOUBLE
    )
""")

print("✅ Tables created successfully.")

# Insert one fake record into activities
con.execute("""
    INSERT INTO activities (activity_id, name, date, distance, duration, fit_filename)
    VALUES (1, 'Morning Run', NOW(), 5.0, 30.0, 'run01.fit')
""")

# Insert one fake record into fit_logs
con.execute("""
    INSERT INTO fit_logs (log_id, activity_id, timestamp, field_name, field_value)
    VALUES (1, 1, NOW(), 'heart_rate', 145)
""")

print("✅ Inserted example data.")

# Query and print
df = con.execute("""
    SELECT a.name, f.timestamp, f.field_name, f.field_value
    FROM activities a
    JOIN fit_logs f ON a.activity_id = f.activity_id
""").fetchdf()

print("\n🎯 Query result:")
print(df)

con.close()


In [ ]:
df